Trabalho 1 - Introdução ao Aprendizado de Máquina
Sérgio Hiroshi Honda - DRE:120035741

Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

Importando os arquivos .csv dos dados de treino e de teste em forma de DataFrames, e retirando a primeira linha de ambos (Chaves Primárias)

In [2]:
dados_treinamento = pd.read_csv("conjunto_de_treinamento.csv")
dados_teste = pd.read_csv("conjunto_de_teste.csv")
exemplo = pd.read_csv("exemplo_arquivo_respostas.csv")

dados_treinamento = dados_treinamento.iloc[:,1:]
dados_teste = dados_teste.iloc[:,1:]

In [3]:
dados_treinamento

,tipo,bairro,tipo_vendedor,quartos,suites,vagas,area_util,area_extra,diferenciais,churrasqueira,estacionamento,piscina,playground,quadra,s_festas,s_jogos,s_ginastica,sauna,vista_mar,preco
0,Casa,Imbiribeira,Imobiliaria,3,3,5,223,167,piscina e copa,0,0,1,0,0,0,0,0,0,0,1000000.0
1,Apartamento,Casa Amarela,Imobiliaria,4,4,2,157,0,piscina e churrasqueira,1,0,1,0,0,0,0,0,0,0,680000.0
2,Apartamento,Encruzilhada,Imobiliaria,3,1,0,53,0,nenhum,0,0,0,0,0,0,0,0,0,0,450000.0
3,Apartamento,Boa Viagem,Imobiliaria,4,3,2,149,0,piscina e churrasqueira,1,0,1,0,0,0,0,0,0,0,1080000.0
4,Apartamento,Rosarinho,Imobiliaria,2,1,1,54,0,piscina e churrasqueira,1,0,1,0,0,0,0,0,0,0,350000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4678,Apartamento,Boa Viagem,Imobiliaria,4,4,3,170,0,piscina e churrasqueira,1,0,1,0,0,0,0,0,0,0,1200000.0
4679,Apartamento,Setubal,Imobiliaria,2,0,1,82,0,churrasqueira,1,0,0,0,0,0,0,0,0,0,258000.0
4680,Apartamento,Torre,Imobiliaria,3,1,1,75,0,nenhum,0,0,0,0,0,0,0,0,0,0,252000.0
4681,Apartamento,Casa Amarela,Imobiliaria,3,2,3,136,0,piscina e copa,0,0,1,0,0,0,0,0,0,0,780000.0


Informações de cada coluna, incluindo os tipos dos dados

In [4]:
dados_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4683 entries, 0 to 4682
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tipo            4683 non-null   object 
 1   bairro          4683 non-null   object 
 2   tipo_vendedor   4683 non-null   object 
 3   quartos         4683 non-null   int64  
 4   suites          4683 non-null   int64  
 5   vagas           4683 non-null   int64  
 6   area_util       4683 non-null   int64  
 7   area_extra      4683 non-null   int64  
 8   diferenciais    4683 non-null   object 
 9   churrasqueira   4683 non-null   int64  
 10  estacionamento  4683 non-null   int64  
 11  piscina         4683 non-null   int64  
 12  playground      4683 non-null   int64  
 13  quadra          4683 non-null   int64  
 14  s_festas        4683 non-null   int64  
 15  s_jogos         4683 non-null   int64  
 16  s_ginastica     4683 non-null   int64  
 17  sauna           4683 non-null   i

Exibindo a quantidade de dados únicos de cada coluna

In [5]:
dados_treinamento.nunique()

tipo                4
bairro             66
tipo_vendedor       2
quartos             9
suites              7
vagas              17
area_util         356
area_extra        170
diferenciais       83
churrasqueira       2
estacionamento      2
piscina             2
playground          2
quadra              2
s_festas            2
s_jogos             2
s_ginastica         2
sauna               2
vista_mar           2
preco             679
dtype: int64

Retirando as colunas bairro (por ser string e não ter nenhum significado númerico) e diferenciais (já está representada por outras colunas existentes) das tabelas. Cria a coluna de tipo_Quitinete na tabela de testes para ficar com as mesmas colunas que a de treinamento.

In [6]:
dados_treinamento.drop(columns=['bairro', 'diferenciais'], inplace=True)
dados_teste.drop(columns=['bairro', 'diferenciais'], inplace=True)
dados_teste['tipo_Quitinete'] = 0

Transformando as colunas que possuem valores de texto em colunas categóricas (númericas), para poderem ser trabalhadas pelos classificadores no final.

In [7]:
dados_treinamento = pd.get_dummies(dados_treinamento, columns=['tipo', 'tipo_vendedor'])
dados_teste = pd.get_dummies(dados_teste, columns=['tipo', 'tipo_vendedor'])

Conferindo se todas as colunas possuem valores numéricos

In [8]:
dados_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4683 entries, 0 to 4682
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   quartos                      4683 non-null   int64  
 1   suites                       4683 non-null   int64  
 2   vagas                        4683 non-null   int64  
 3   area_util                    4683 non-null   int64  
 4   area_extra                   4683 non-null   int64  
 5   churrasqueira                4683 non-null   int64  
 6   estacionamento               4683 non-null   int64  
 7   piscina                      4683 non-null   int64  
 8   playground                   4683 non-null   int64  
 9   quadra                       4683 non-null   int64  
 10  s_festas                     4683 non-null   int64  
 11  s_jogos                      4683 non-null   int64  
 12  s_ginastica                  4683 non-null   int64  
 13  sauna             

Exibe o coeficiente de Pearson de cada coluna na tabela

In [9]:
colunas_treinamento = dados_treinamento.columns
for col in colunas_treinamento:
    print('%10s = %6.5f' % ( col , pearsonr(dados_treinamento[col],dados_treinamento['preco'])[0] ) )

   quartos = 0.02089
    suites = 0.04213
     vagas = 0.02905
 area_util = 0.02756
area_extra = 0.00114
churrasqueira = 0.01588
estacionamento = -0.00637
   piscina = 0.00681
playground = -0.01220
    quadra = -0.00439
  s_festas = 0.00754
   s_jogos = -0.00254
s_ginastica = -0.00230
     sauna = 0.00117
 vista_mar = 0.00766
     preco = 1.00000
tipo_Apartamento = -0.00854
 tipo_Casa = 0.00900
 tipo_Loft = -0.00142
tipo_Quitinete = -0.00150
tipo_vendedor_Imobiliaria = -0.12405
tipo_vendedor_Pessoa Fisica = 0.12405


Cria dois arrays ('x' e 'y'), de forma que o segundo possui apenas a coluna target (inadimplente) e o primeiro possui todas as outras. Em seguida, usa-se a função train_test_split para separar os arrays de treino e os de teste, que serão usados no modelo, com a proporção de 70% e 30%, respectivamente.

In [10]:
from sklearn.model_selection import train_test_split

x = dados_treinamento.drop(columns='preco').values
y = dados_treinamento['preco'].values

x_treino, x_teste, y_treino, y_teste = train_test_split(
        x,
        y,
        test_size =3300,
        random_state = 13
)

Cria um scaler para normalizar os dados dos arrays de treinamento e de teste, de forma que nenhuma coluna possua um peso maior do que a outra a priori.

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler  = MinMaxScaler()
x_treino = scaler.fit_transform(x_treino)
x_teste  = scaler.transform(x_teste)

scaler = MinMaxScaler()
x = scaler.fit_transform(x)
dados_teste = scaler.transform(dados_teste)

C:\Users\sergi\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Criação de uma função para exibir os melhores parâmetros de cada modelo (RandomForest, KNN e GradientBoost).

In [12]:
def exibicao(frame):
    resultado = pd.DataFrame(frame.cv_results_)
    resultado.sort_values(by='rank_test_score', inplace=True)
    resultado = resultado[['params','mean_test_score','std_test_score']].head()

    return resultado

Criação e utilização dos 3 regressores, com a utilização da função GridSearchCV para encontrar os melhores modelos para cada algoritmo. A precisão utilizada será igual a do trabalho, o RMSE.

In [13]:
# RandomForest

parametrosRF = {
    "max_depth":[2,4,10],
    "n_estimators": [500],
    "min_samples_leaf":[2,5,8],
    "criterion":['absolute_error'],
    }

exibRF = GridSearchCV(RandomForestRegressor(), parametrosRF,  scoring='neg_mean_squared_error',cv=2, verbose=2, n_jobs =-1)
exibRF.fit(x, y)

bestRF = exibRF.best_estimator_
bestRF.fit(x,y)
respostaRF = bestRF.predict(dados_teste)


# KNN

parametrosKNN = {
    "p": [1,2],
    "n_neighbors": list(range(200,851,20)),
    'weights': ['uniform'],
    'algorithm':['auto']
    }

exibknn = GridSearchCV(KNeighborsRegressor(), parametrosKNN, scoring='neg_mean_squared_error', cv=35, verbose=2, n_jobs =-1)
exibknn.fit(x,y)

bestknn = exibknn.best_estimator_
bestknn.fit(x,y)
respostaknn = bestknn.predict(dados_teste)

# Gradient Boost Classifier

parametrosGB = {
    'learning_rate': [0.1,0.01,0.001],
    "n_estimators": [750],
    'max_depth': [1,4,7,10,13]
    }

exibGB = GridSearchCV(GradientBoostingRegressor(), parametrosGB, scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs =-1)
exibGB.fit(x, y)

bestGB = exibGB.best_estimator_
bestGB.fit(x,y)
respostaGB = bestGB.predict(dados_teste)


Fitting 2 folds for each of 9 candidates, totalling 18 fits
Fitting 35 folds for each of 66 candidates, totalling 2310 fits
Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [14]:
exibicao(exibRF)

,params,mean_test_score,std_test_score
7,"{'criterion': 'absolute_error', 'max_depth': 1...",-1.102087e+14,5.905935e+13
8,"{'criterion': 'absolute_error', 'max_depth': 1...",-1.102122e+14,5.906250e+13
4,"{'criterion': 'absolute_error', 'max_depth': 4...",-1.102131e+14,5.905898e+13
5,"{'criterion': 'absolute_error', 'max_depth': 4...",-1.102140e+14,5.906016e+13
2,"{'criterion': 'absolute_error', 'max_depth': 2...",-1.102322e+14,5.902096e+13


In [15]:
exibicao(exibknn)

,params,mean_test_score,std_test_score
21,"{'algorithm': 'auto', 'n_neighbors': 400, 'p':...",-1.103459e+14,5.087398e+14
19,"{'algorithm': 'auto', 'n_neighbors': 380, 'p':...",-1.103466e+14,5.087218e+14
17,"{'algorithm': 'auto', 'n_neighbors': 360, 'p':...",-1.103476e+14,5.087036e+14
11,"{'algorithm': 'auto', 'n_neighbors': 300, 'p':...",-1.103490e+14,5.086329e+14
15,"{'algorithm': 'auto', 'n_neighbors': 340, 'p':...",-1.103496e+14,5.086841e+14


In [16]:
exibicao(exibGB)

,params,mean_test_score,std_test_score
5,"{'learning_rate': 0.01, 'max_depth': 1, 'n_est...",-1.097099e+14,1.610527e+14
10,"{'learning_rate': 0.001, 'max_depth': 1, 'n_es...",-1.097214e+14,1.621043e+14
0,"{'learning_rate': 0.1, 'max_depth': 1, 'n_esti...",-1.103310e+14,1.608238e+14
11,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",-1.112038e+14,1.633459e+14
13,"{'learning_rate': 0.001, 'max_depth': 10, 'n_e...",-1.121729e+14,1.625972e+14


Já que a função GridSearch utilizada considera o melhor modelo o que possuir a menor pontuação, seu algoritmo precisa inverter o resultado para conseguirmos analisar métricas com o menor resultado possível. Por isso, as RMSE estão negativas, e os modelos do topo continuam sendo os melhores.

Analisando os resultados obtidos, o que apresentou melhor desempenho foi o RandomForest, por possuir a menor pontuação.

In [17]:
resposta = respostaRF

exemplo['preco'] = respostaRF
exemplo.to_csv('resposta.csv', index= False)